In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### pytorch数据集 神经网络搭建和训练

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# 超参数设置
LR = 1e-3
EPOCHS = 20
BATCH_SIZE = 32

# 数据预处理与加载
# ------------------------------------------------
# 加载Olivetti Faces数据集
olivetti = fetch_olivetti_faces()
data = olivetti.data.astype(np.float32)  # 转换为float32类型
targets = olivetti.target

# 按类别划分训练集（每个类前7个样本）和测试集（每个类后3个样本）
train_indices, test_indices = [], []
for i in range(40):
    train_indices.extend(range(i*10, i*10+7))
    test_indices.extend(range(i*10+7, (i+1)*10))


X_train, y_train = data[train_indices], targets[train_indices]
X_test, y_test = data[test_indices], targets[test_indices]

# 数据标准化（归一化）
mean = X_train.mean(axis=0)
std = X_train.std(axis=0) + 1e-8  # 防止除零
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# 自定义Dataset类
class OlivettiDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 创建DataLoader
train_dataset = OlivettiDataset(X_train, y_train)
test_dataset = OlivettiDataset(X_test, y_test)
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE)


# 模型定义（包含归一化和正则化）
# ------------------------------------------------
class TorchNN(nn.Module):
    def __init__(self, input_size=4096, hidden_size=512, output_size=40):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),      # 批归一化
            nn.ReLU(),
            nn.Dropout(0.5),                  # Dropout正则化
            nn.Linear(hidden_size, hidden_size//2),
            nn.BatchNorm1d(hidden_size//2),   # 批归一化
            nn.ReLU(),
            nn.Dropout(0.3),                 # Dropout正则化
            nn.Linear(hidden_size//2, output_size)
        )
    
    def forward(self, x):
        return self.net(x)

# 训练与测试函数
# ------------------------------------------------
def train_model(model, optimizer, loss_fn, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X_batch.size(0)
        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

def test_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    print(f'Test Accuracy: {100 * correct/total:.2f}%')

# 实验：不同优化器对比
# ------------------------------------------------
optimizers = {
    'Adam': optim.Adam,
    'SGD': optim.SGD,
    'RMSprop': optim.RMSprop
}

# 配置不同优化器参数
optimizer_configs = {
    'Adam': {'lr': LR, 'weight_decay': 1e-4},          # L2正则化
    'SGD': {'lr': LR, 'momentum': 0.9, 'weight_decay': 1e-4},
    'RMSprop': {'lr': LR, 'weight_decay': 1e-4}
}

# 遍历所有优化器进行实验
for opt_name in optimizers:
    print(f"\n=== 当前优化器：{opt_name} ===")
    model = TorchNN()
    loss_fn = nn.CrossEntropyLoss()
    
    # 初始化优化器
    if opt_name == 'SGD':
        optimizer = optimizers[opt_name](model.parameters(), **optimizer_configs[opt_name])
    else:
        optimizer = optimizers[opt_name](model.parameters(), lr=LR, weight_decay=1e-4)
    
    # 训练与验证
    train_model(model, optimizer, loss_fn, train_dl, EPOCHS)
    test_model(model, test_dl)


=== 当前优化器：Adam ===
Epoch [1/20], Loss: 3.4103
Epoch [2/20], Loss: 2.6345
Epoch [3/20], Loss: 2.1163
Epoch [4/20], Loss: 1.7481
Epoch [5/20], Loss: 1.3962
Epoch [6/20], Loss: 1.1046
Epoch [7/20], Loss: 0.8600
Epoch [8/20], Loss: 0.7320
Epoch [9/20], Loss: 0.5477
Epoch [10/20], Loss: 0.4458
Epoch [11/20], Loss: 0.3364
Epoch [12/20], Loss: 0.2743
Epoch [13/20], Loss: 0.2304
Epoch [14/20], Loss: 0.1984
Epoch [15/20], Loss: 0.1686
Epoch [16/20], Loss: 0.1401
Epoch [17/20], Loss: 0.1250
Epoch [18/20], Loss: 0.1176
Epoch [19/20], Loss: 0.1027
Epoch [20/20], Loss: 0.0780
Test Accuracy: 97.50%

=== 当前优化器：SGD ===
Epoch [1/20], Loss: 3.7121
Epoch [2/20], Loss: 3.3841
Epoch [3/20], Loss: 3.1049
Epoch [4/20], Loss: 2.8290
Epoch [5/20], Loss: 2.6075
Epoch [6/20], Loss: 2.4211
Epoch [7/20], Loss: 2.2787
Epoch [8/20], Loss: 2.1859
Epoch [9/20], Loss: 2.0270
Epoch [10/20], Loss: 1.9377
Epoch [11/20], Loss: 1.8155
Epoch [12/20], Loss: 1.7443
Epoch [13/20], Loss: 1.6388
Epoch [14/20], Loss: 1.5394
Epoch